In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
data = pd.read_csv('data/tictactoe_games.csv')

columns = ['Winner', 'Move1_X', 'Move2_O', 'Move3_X', 'Move4_O', 'Move5_X']
df = pd.DataFrame(data, columns=columns)


,Winner,Move 1-X (Row-Col),Move 2-O (Row-Col),Move 3-X (Row-Col),Move 4-O (Row-Col),Move 5-X (Row-Col),Move 6-O (Row-Col),Move 7-X (Row-Col),Move 8-O (Row-Col),Move 9-X (Row-Col)
0,X,0-0,0-1,1-0,0-2,2-0,---,---,---,---
1,X,0-0,0-1,1-0,1-1,2-0,---,---,---,---
2,X,0-0,0-1,1-0,1-2,2-0,---,---,---,---
3,X,0-0,0-1,1-0,2-1,2-0,---,---,---,---
4,X,0-0,0-1,1-0,2-2,2-0,---,---,---,---
